In [ ]:
%matplotlib inline
#%matplotlib notebook

For high dpi displays.

In [ ]:
%config InlineBackend.figure_format = 'retina'

# 0. General note

- Loads in data from single LOG files.
- Loads in ruby spectrum .asc files.
- Fits ruby spectrum with 2 Lorentzians and a linear background.
- Hannah Bausch - 171212

# 1. General setup

In [ ]:
import os
import sys
import numpy as np
from scipy import constants as con
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import uncertainties as uct
from uncertainties import unumpy as unp
from lmfit import Model
from lmfit.models import LinearModel, LorentzianModel
import pandas as pd
from IPython.display import display
import burnman
import glob
import re

### What sample is this? - Where are the data folders?

In [ ]:
sample = 'Acetaminophen'
T = 298.15

ruby_path = r'C:\Users\Huber\Box Sync\X-ray\acetaminophen\ruby'
log_path = r'C:\Users\Huber\Box Sync\X-ray\acetaminophen'

if os.path.exists(os.path.abspath(ruby_path)):
    ruby_folder = os.path.abspath(ruby_path)
    log_folder = os.path.abspath(log_path)
    print('Using Huber working directory.')
else:
    ruby_path = r'C:\data\Raman\Bausch\acetaminophen\after'
    log_path = r'C:\data\Raman\Bausch\acetaminophen\after'
    
    if os.path.exists(os.path.abspath(ruby_path)):
        ruby_folder = os.path.abspath(ruby_path)
        log_folder = os.path.abspath(log_path)
        print('Using Raman working directory.')
    else:
        print('Directory does not exist.......')


# 2. Read data

Load in the .log files from directory as well as ruby .asc files from a folder named "ruby" within the directory.


In [ ]:
#idx_file = 0
#sample = logs[idx_file]
os.chdir(ruby_folder)
rubys = glob.glob('*.asc')
print('%s\n' % rubys)
print(sample)

# 3. Organize and fit data

First, we sort the ruby spectra into 2 lists. One list for 'table_rubys' and one list for 'sample_rubys'.

#### extract_ruby
In the 'extract_ruby' function we load the ruby spectrum from '.asc' files into lists. We then fit the data in the 'ruby_fit' funciton using the (Dawaele et al. 2008) equation to determine pressure. 


In [ ]:
table_rubys = []
sample_rubys = []
for i in  rubys:
    if re.search('table',i):
        table_rubys.append(i)
    else:
        sample_rubys.append(i)

In [ ]:
def calc_pressure(sample_lam,table_lam):
    a = 1920.0
    b = 9.61
    r1 = sample_lam
    r1_o = table_lam
    p = (a/b)*(((r1/r1_o)**b)-1)
    return p

def normalize(y):
    z = (y-np.min(y))/(np.max(y)-np.min(y))
    return z

def ruby_fit(x,y_data,ruby):
    #y = normalize(y_data)
    y = y_data
    background  = LinearModel(prefix='lin_')
    pars = background.guess(y, x=x)

    r1 = LorentzianModel(prefix='r1_')
    pars.update(r1.make_params())
    pars['r1_center'].set(694.3)
    #pars['r1_sigma'].set(0.2)
    r2 = LorentzianModel(prefix='r2_')
    pars.update(r2.make_params())

    pars['r2_center'].set(692.9)
    #pars['r2_sigma'].set(0.2)
    mod = r1 + r2 + background
    init = mod.eval(pars, x=x)
    plt.plot(x, y)

    out = mod.fit(y, pars, x=x)

    comps = out.eval_components(x=x)

    print(out.fit_report(min_correl=0.5))

    plt.plot(x, out.best_fit, 'r-')
    plt.plot(x, comps['r1_'], 'b--')
    plt.plot(x, comps['r2_'], 'b--')
    plt.plot(x, comps['lin_'], 'k--')
    plt.title(ruby,fontweight='bold')
    plt.xlabel('Wavelength (nm)',fontweight='bold')
    plt.ylabel('Intensity (arb. units)',fontweight='bold')

    r1_center = str(out.params['r1_center']).split()[2]
    r1_center = float(r1_center.strip('value='))
    r1_esd = str(out.params['r1_center']).split()[4]
    r1_esd = float(r1_esd.strip(','))
    r2_center = str(out.params['r2_center']).split()[2]
    r2_center = float(r2_center.strip('value='))
    r2_esd = str(out.params['r2_center']).split()[4]
    r2_esd = float(r2_esd.strip(','))

    ruby_cent = [r1_center,r2_center]
    ruby_esd = [r1_esd,r2_esd]
    #print(ruby_cent)
    #print(ruby_esd)
    return ruby_cent

def extract_ruby(ruby):    
    with open(ruby, "r") as f:
        data = np.loadtxt(f)

    x = data[:,0]
    y = data[:,1]
    
    y_norm = normalize(y)
    return x,y_norm

#table ruby wavelength
table_lam = []
for i in table_rubys:
    fig = plt.figure()
    x_table,y_table = extract_ruby(i)
    r1_table,r2_table = ruby_fit(x_table,y_table,i)
    table_lam.append(r1_table)
    
sample_lam = []
for j in sample_rubys:
    fig = plt.figure()
    x,y = extract_ruby(j)
    r1,r2 = ruby_fit(x,y,j)
    sample_lam.append(r1)

dac_pressures = []
for i in range(len(sample_lam)):
    p = calc_pressure(sample_lam[i],table_lam[i])
    dac_pressures.append(p)
    
if dac_pressures[0] == 0:
    dac_pressures[0] = 1e-8

print(dac_pressures)

In [ ]:
print('PRESSURES: \n')
for i in range(len(dac_pressures)): 
    print('P%s - %.3f GPa' % (i,dac_pressures[i]))